In [1]:
import json
import foursquareWithHosts
with open('keys/appCredentialsSudo.json') as credentialsFile:    
    credentials = json.load(credentialsFile)
with open('keys/hosts.json') as hostsFile:    
    hosts = json.load(hostsFile)
with open('keys/accessTokensSudo.json') as tokensFile:    
    accessTokens = json.load(tokensFile)

In [34]:
from datetime import datetime
from sys import stdout

In [2]:
clients = []
users = []
for accessToken in accessTokens[:5]:
    client = foursquareWithHosts.Foursquare(client_id=credentials['client_id'], client_secret=credentials['client_secret'], redirect_uri=credentials['redirect_uri'])
    client.set_access_token(accessToken)
    user = client.users()
    clients.append(client)
    users.append(user)

Hi
Hi

/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning



Hi
Hi

/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning



Hi


/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [ ]:
auth_uri = client.oauth.auth_url()

In [ ]:
# Interrogate foursquare's servers to get the user's access_token
access_token = client.oauth.get_token('code')

# Apply the returned access token to the client
client.set_access_token(access_token)

# Get the user's data
user = client.users()

In [ ]:
client.set_access_token(accessTokens[1])

# Get the user's data
user = client.users()

In [ ]:
#1 Get the venues of interes
yogaVenues = client.venues.search(params={'query': 'worldclass', 'near':'Moscow'})
for yogaPlace in yogaVenues['venues']:
    if yogaPlace['hereNow']['count'] > 0:
        print ''
        print yogaPlace['name']
        print yogaPlace['id']
        print 'here now:', yogaPlace['hereNow']['count']
        print 'total users:', yogaPlace['stats']['usersCount']

In [35]:
print yogaVenues['venues'][0].keys()

[u'verified', u'name', u'referralId', u'hereNow', u'specials', u'contact', u'location', u'stats', u'id', u'categories']


In [14]:
#Save info about Moscow
with open('moscowGeocode.json', 'w') as outfile:
    json.dump(yogaVenues['geocode'], outfile)

In [ ]:
#2 Check-in a user into the venue of interest
clients[0].checkins.add({'venueId':venuesForUsers[0][0]})

In [ ]:
#3 Get info about the venue right now
hereNowDict = clients[0].venues.herenow(venuesForUsers[0][0])

# Getting the data 

In [18]:
import pandas as pd
usersDF = pd.DataFrame()
checkinsDF = pd.DataFrame()
checkpointsDF = pd.DataFrame()
allUsersIds = []
allCheckInsIds = []

currentVenues = [0,0,0]
currentBotId = 0

In [19]:
#3.1 Savethe info to the DB
def parseHereNowDict(hereNowDict, venueId):
    usersInVenue = []
    currentCheckIns = []

    if hereNowDict['hereNow']:
        global checkpointsDF
        global usersDF
        global checkinsDF
        hereNowCount =  hereNowDict['hereNow']['count']
        checkpointsDF = checkpointsDF.append([{'createdAt':datetime.now(),
                                               'hereNow':hereNowCount, 
                                               'venueId':venueId}])

        print 'Here now:', hereNowCount
        print ''

        #Get check-in info first
        for checkinAtVenue in hereNowDict['hereNow']['items']:
            if checkinAtVenue['type'] == 'checkin':
                if checkinAtVenue['id'] not in allCheckInsIds:
                    currentCheckIns.append({'checkInId':checkinAtVenue['id'],
                                            'createdAt':checkinAtVenue['createdAt'],
                                            'createdAtDate':extractComponents(checkinAtVenue['createdAt'],checkinAtVenue['timeZoneOffset']),
                                            'venueId':venueId,
                                            'userId':checkinAtVenue['user']['id']
                                           })
                    allCheckInsIds.append(checkinAtVenue['id'])
                    #Now get the user
                    userInVenue =  checkinAtVenue['user']
                    #print 'User', userInVenue['id'],'Checked-in at:', extractComponents(checkinAtVenue['createdAt'],checkinAtVenue['timeZoneOffset'])
                    if userInVenue['id'] not in allUsersIds:
                        userInfo = {'userId':userInVenue['id']}
                        if 'gender' in userInVenue.keys() and userInVenue['gender']:
                            userInfo["gender"] = userInVenue['gender']
                        if 'firstName' in userInVenue.keys() and userInVenue['firstName']:
                            userInfo["firstName"] = userInVenue['firstName']
                        if 'lastName' in userInVenue.keys() and userInVenue['lastName']:
                            userInfo["lastName"] = userInVenue['lastName']
                        usersInVenue.append(userInfo)
                        allUsersIds.append(userInVenue['id'])
        if len(usersInVenue):
            usersDF = usersDF.append(usersInVenue)
        if len(currentCheckIns):
            checkinsDF = checkinsDF.append(currentCheckIns)

        usersDF.to_csv('data/realUsers.tsv', sep='\t',encoding='utf-8')
        checkinsDF.to_csv('data/realCheckins.tsv', sep='\t',encoding='utf-8')
        checkpointsDF.to_csv('data/checkPoints.tsv', sep='\t',encoding='utf-8')

In [20]:
def getNextCheckinInfo():
    global currentBotId
    global currentVenues
    currentVenue = venuesForUsers[currentBotId][currentVenues[currentBotId]]
    checkIn = clients[currentBotId].checkins.add(currentVenue)
    hereNowDict = clients[currentBotId].venues.herenow(currentVenue['venueId'])
    parseHereNowDict(hereNowDict, currentVenue['venueId'])
    
    currentVenues[currentBotId] = (currentVenues[currentBotId]+1)%len(venuesForUsers[currentBotId])
    currentBotId = (currentBotId+1)%len(clients)
    print currentBotId
    print currentVenues

In [21]:
with open('data/top90healthVenuesSummer.json') as venuesFile:
    top50HealtPlaces = json.load(venuesFile)
    
venuesForUsers = [[],[],[]]
for i, x in enumerate(top50HealtPlaces):
    venuesForUsers[i%3].append(x)

In [164]:
from datetime import datetime
def extractComponents(createdAt, tzOffset):
    date = datetime.utcfromtimestamp(createdAt+(tzOffset-60)*60)
    return date

In [8]:
timeInterval = 3*60/(len(venuesForUsers[0])*len(clients))

In [ ]:
import schedule
import time

timeInterval = 3*60/(len(venuesForUsers[0])*len(clients))
schedule.every(1).minutes.do(getNextCheckinInfo)

while True:
    schedule.run_pending()
    time.sleep(1)

In [44]:
checkinsDF1.to_csv('data/realCheckins.tsv', sep='\t',encoding='utf-8')

# Getting Friends

In [75]:
from datetime import datetime
from sys import stdout
import pandas as pd

In [151]:
checkinsDF = pd.read_csv('data/24June2/checkins.tsv', sep='\t',encoding='utf-8')
checkpointsDF = pd.read_csv('data/24June2/checkPoints.tsv', sep='\t',encoding='utf-8')
usersDF = pd.read_csv('data/24June2/users.tsv', sep='\t',encoding='utf-8')

In [ ]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.habidatum

In [152]:
#Getting old requested and pending users
existingRequestedUsers = []
existingRequestedUsersCoursor = db.requestedUsers.find()
for requestedUser in existingRequestedUsersCoursor:
    existingRequestedUsers.append(int(requestedUser['requestedUser']['user']['id']))

existingPendingFriends = []
existingPendingFriendsCoursor = db.pendingUsers.find()
for pendingFriend in existingPendingFriendsCoursor:
    existingPendingFriends.append(int(pendingFriend['requestedUser']['user']['id']))

In [153]:
usersToRequest = usersDF[~usersDF.userId.isin(existingRequestedUsers)].userId.tolist()
usersToFriend = usersDF[~usersDF.userId.isin(existingPendingFriends)].userId.tolist()

In [162]:
len(usersToRequest)

237

In [ ]:
requestedUsers = []
for index, userId in enumerate(usersToRequest):
    progress(index,len(usersToRequest))
    requestedUser = clients[index%len(clients)].users(userId)
    requestedUsers.append(requestedUser)
    db.requestedUsers.insert({'requestingUser':index%len(clients),
                                'requestedUser':requestedUser,
                                 'requestedAt':datetime.now()})

In [163]:
pendingUsers = []
for index, userId in enumerate(usersToFriend):
    progress(index,len(usersToFriend))
    pendingUser = clients[index%len(clients)].users.request(userId)
    pendingUsers.append(pendingUser)
    db.pendingUsers.insert({'requestingUser':index%len(clients),
                                'requestedUser':pendingUser,
                                 'requestedAt':datetime.now()})

100%



In [ ]:
allRequestedStoredUsers = db.pendingUsers.find({"requestingUser":3},{"_id":0})
for userR in allRequestedStoredUsers:
    print userR["requestedUser"]["user"]["checkins"]["count"]

In [ ]:
#4 Get the user that was checked-in
requestedUser = client.users('someUserId')

#5 Send requests to their friends 
if requestedUser['user']:
    requestedUserInfo = requestedUser['user']
    print 'Blocked:', requestedUserInfo['blockedStatus']
    print requestedUserInfo['homeCity']
    print requestedUserInfo['checkins']
    print requestedUserInfo['gender']
    print 'friends count:', requestedUserInfo['friends']['count']
    for friendGroup in requestedUserInfo['friends']['groups']:
        if friendGroup['type'] == 'friends':
            if friendGroup['count'] > 3:
                print 'Enough mutual friends'
                break
        if 0:
            print 'Already requested'
        print ''
        for friend in friendGroup['items']:
            if not friend['id'] in requestedUsersFriends:
                print 'Add'
                requestedUsersFriends.append(friend['id'])
                client.users.request(friend['id'])
    #0 for Mutual friends, type = 'friends', 1 for other friends, type = 'others'
    #print requestedUserInfo['friends']['groups']
    #print requestedUser['user']['lists']
     #print requestedUser['user']['tips']

In [66]:
requestedUser['user'].keys()

[u'checkins',
 u'bio',
 u'gender',
 u'type',
 u'firstName',
 u'lastName',
 u'blockedStatus',
 u'pings',
 u'homeCity',
 u'photos',
 u'contact',
 u'checkinPings',
 u'lists',
 u'photo',
 u'tips',
 u'mayorships',
 u'friends',
 u'id',
 u'badges']

In [179]:
print user['user']['friends']['groups'][1]

{u'count': 0, u'items': [], u'type': u'others', u'name': u'Other friends'}


In [128]:
# Get the user's data
user = client.users()

In [ ]:
#6 One way to get some check-in
aFriend = client.users('')
print aFriend['user']['checkins']['items'][0]

In [180]:
#6 Another way — get recent check-ins
recent = client.checkins.recent()['recent']

# Get Activities

In [28]:
def progress(i, n):
    stdout.write("\r%f%%" % (i*100/float(n)))
    stdout.flush()
    if i == n-1:
        stdout.write("\r100%")
        print("\r\n")

In [237]:
#Get current friends info
usersFriends = []
for client in clients[:5]:
    userInfo = client.users()
    usersFriends.append(userInfo['user']['friends'])

In [238]:
#Get current friends ids
avaliableFriends = []
for userFriendSet in usersFriends:
    userFriendGroup = userFriendSet['groups'][1]['items']
    friendIds = []
    for friend in userFriendGroup:
        friendIds.append(friend['id'])
    avaliableFriends.append(friendIds)
print(avaliableFriends)

[[u'88228148', u'34214626', u'69456382', u'84581388', u'39412244', u'74434491', u'36992706', u'88664435', u'22067849', u'59201697'], [u'28817862', u'50319940', u'6186570', u'31333619', u'52043686', u'39412205', u'64994770', u'128636353'], [u'79385836', u'77922328', u'97221369', u'86408982', u'65982110', u'40653286', u'7369996', u'44738421', u'5599388', u'58243642'], [u'12700194', u'25345770', u'58092923', u'64994791', u'13031153', u'69499174', u'86717627', u'131302695', u'130936768'], [u'67732190', u'46916093', u'116444949', u'55049378', u'63842094', u'61474907', u'13886058', u'39702002', u'126472040', u'91418757']]


In [227]:
print len(requestedActivitiesIds)

51


In [226]:
userActivities = []

for index, userSet in enumerate(avaliableFriends):
    progress(index, len(clients))
    for userId in userSet:
        if userId not in requestedActivitiesIds:
            userActivity = clients[index].users.activities(userId)
            userActivities.append(userActivity)
            db.activities.insert({'requestingUser':index,
                                    'requestedUserId':userId,
                                    'activity':userActivity,
                                    'requestedAt':datetime.now()})
            requestedActivitiesIds.append(userId)

100%



In [231]:
activityProjection = {"_id":0,
                      "activity.activities.items.content.object.venue.id":1,
                      "activity.activities.items.content.object.venue.name":1,
                        "activity.activities.items.content.object.venue.location.lat":1,
                        "activity.activities.items.content.object.venue.location.lng":1,
                        "activity.activities.items.content.object.checkin.venue.id":1,
                        "activity.activities.items.content.object.checkin.venue.name":1,
                         "activity.activities.items.content.object.checkin.venue.location.lat":1,
                         "activity.activities.items.content.object.checkin.venue.location.lng":1,
                         "activity.activities.items.createdAt":1,
                          "requestedUserId":1}
activityQuery = {"activity.activities.items.content.type":"checkin"}
usersVenues = db.activities.find(activityQuery, activityProjection)

In [232]:
checkinsForUsers = []
for userActivity in enumerate(usersVenues):
    checkinsForUser = []
    for checkIn in userActivity[1]['activity']['activities']['items']:
        checkinInfo = {}
        checkinInfo['userId'] = userActivity[1]['requestedUserId']
        checkinInfo["createdAt"] = checkIn['createdAt']
        checkinInfo["createdAtDate"] = str(extractComponents(checkIn['createdAt'], 240))#Get the real time zone
        
        if "venue" in checkIn['content']['object'].keys():
            checkinInfo["venueId"] = checkIn['content']['object']["venue"]["id"]
            checkinInfo["name"] = checkIn['content']['object']["venue"]["name"]
            checkinInfo["lat"] = checkIn['content']['object']["venue"]["location"]["lat"]
            checkinInfo["lng"] = checkIn['content']['object']["venue"]["location"]["lng"]
            checkinsForUser.append(checkinInfo)
        elif "venue" in checkIn['content']['object'].keys():
            checkinInfo["venueId"] = checkIn['content']['object']['checkin']["venue"]["id"]
            checkinInfo["name"] = checkIn['content']['object']['checkin']["venue"]["name"]
            checkinInfo["lat"] = checkIn['content']['object']['checkin']["venue"]["location"]["lat"]
            checkinInfo["lng"] = checkIn['content']['object']['checkin']["venue"]["location"]["lng"]
            checkinsForUser.append(checkinInfo)
    checkinsForUsers.append(checkinsForUser)

In [235]:
newIds = []
for userSet in avaliableFriends:
    for user in userSet:
        newIds.append(user)

In [160]:
import requests
requests.packages.urllib3.disable_warnings()

In [233]:
plainListOfCheckins = [item for sublist in checkinsForUsers for item in sublist]
plainListOfCheckinsDF = pd.DataFrame(plainListOfCheckins)
plainListOfCheckinsDF.to_csv('data/24June2/checkinsHistory.tsv', sep='\t',encoding='utf-8')

In [234]:
len(plainListOfCheckins)

6529

In [239]:
len(newIds)

46